In [1]:
from random import seed
from random import random
from math import exp
import numpy as np
import pandas as pd
import textwrap
import timeit
import binascii
import json

In [2]:
# -*- coding: utf-8 -*-
import json

# Make it work for Python 2+3 and with Unicode
import io
try:
    to_unicode = unicode
except NameError:
    to_unicode = str

In [3]:
def addZeros(strr, n): 
    for i in range(n): 
        strr = "0" + strr 
    return strr 

def getXOR(a, b): 
    aLen = len(a) 
    bLen = len(b) 


    if (aLen > bLen): 
        b = addZeros(b, aLen - bLen) 
    elif (bLen > aLen): 
        a = addZeros(a, bLen - aLen) 
 
    lenn = max(aLen, bLen); 

    res = "" 
    for i in range(lenn): 
        if (a[i] == b[i]): 
            res += "0"
        else: 
            res += "1"
    return res 

In [4]:
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

In [5]:
#initializing the network
def initialize_network(parameters):
    network = list()
    for j in range(1,len(parameters)):
        layer = [{'weights':[random() for i in range(parameters[j-1] + 1)]} for i in range(parameters[j])]
        network.append(layer)
    return network

In [6]:
# Make a prediction with a network
def predict(network, row, encr):
    outputs = forward_propagate(network, row, encr)
    k = [round(i) for i in outputs]
    return k

In [7]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [8]:
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [9]:
# Forward propagate input to a network output
def forward_propagate(network, row, getEncryption):
    count = 0
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        if count == 1 and getEncryption == 1:
            print("io")
            return new_inputs
        inputs = new_inputs
        count += 1
  
    return inputs

In [10]:
# Update network weights with error
def update_weights(network, row, l_rate):
    
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    count = 1
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            count += 1
            outputs = forward_propagate(network, row, 0)
            expected = row[-1]
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        if count % 100 == 0:
            print('> epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        count += 1

In [11]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [12]:
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [13]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    #print(inputs)
    activation = weights[-1]
    for i in range(len(weights)-1):
        #print(i)
        #print(type(weights[i]))
        #print(type(inputs[i]))
        activation += weights[i] * inputs[i]
    return activation

In [14]:
#batch training
# Train a network for a fixed number of epochs
def train_network_batch(network, train, l_rate, n_epoch, n_outputs):
    count = 1
    k = 0
    batch = 2
    for i in range(batch):
        print('> Batch = %d' % (i+1))
        for epoch in range(n_epoch):
            sum_error = 0
            for row in train[k:k+10]:
                count += 1
                outputs = forward_propagate(network, row, 0)
                expected = row[-1]
                sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
                backward_propagate_error(network, expected)
                update_weights(network, row, l_rate)
            if count % 100 == 0:
                print('    epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
            count += 1
        k += 10

In [87]:
# Read JSON file
with open('../network/network.json') as data_file:
    network = json.load(data_file)

In [88]:
# Read JSON file
with open('../datasets/languageDecryptor.json') as data_file:
    dataset = json.load(data_file)

with open('../datasets/languageDecryptor.json') as data_file:
    temp = json.load(data_file)

In [76]:
for i in range(100):
    k = temp[i][-1]
    k.append(temp[i][:-1])
    dataset.append(k)

In [102]:
dataset

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0, 1, 1, 0, 0, 0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0, 0, 1, 1, 1, 0, 0, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0, 0, 1, 1, 1, 0, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 1, 1, 0, 0, 0, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 1, 1, 0, 0, 0, 1, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1, 1, 0, 0, 1, 0, 0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 0, 1, 1, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 1, 0, 0, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 0, 0

In [116]:
seed(1)
network = initialize_network([8,16,16,8])

In [117]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(dataset[0][-1])
start = timeit.default_timer()
train_network(network, dataset[0:25], 0.5, 10000, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

> epoch=49, lrate=0.500, error=72.857
> epoch=99, lrate=0.500, error=50.117
> epoch=149, lrate=0.500, error=50.113
> epoch=199, lrate=0.500, error=50.043
> epoch=249, lrate=0.500, error=49.632
> epoch=299, lrate=0.500, error=44.744
> epoch=349, lrate=0.500, error=45.690
> epoch=399, lrate=0.500, error=44.900
> epoch=449, lrate=0.500, error=44.312
> epoch=499, lrate=0.500, error=44.563
> epoch=549, lrate=0.500, error=44.532
> epoch=599, lrate=0.500, error=44.539
> epoch=649, lrate=0.500, error=37.106
> epoch=699, lrate=0.500, error=31.854
> epoch=749, lrate=0.500, error=25.264
> epoch=799, lrate=0.500, error=18.798
> epoch=849, lrate=0.500, error=13.427
> epoch=899, lrate=0.500, error=11.364
> epoch=949, lrate=0.500, error=9.986
> epoch=999, lrate=0.500, error=6.818
> epoch=1049, lrate=0.500, error=4.119
> epoch=1099, lrate=0.500, error=3.606
> epoch=1149, lrate=0.500, error=3.430
> epoch=1199, lrate=0.500, error=3.336
> epoch=1249, lrate=0.500, error=3.275
> epoch=1299, lrate=0.500, er

In [112]:
c = 0
for i in dataset:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        c += 1
print(str(c)+"% error")

73% error


In [136]:
c = 0
new_dataset = list()
for i in dataset:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        new_dataset.append(i)
        print("index = %d" % (c))
        print(k)
        print(j)
        print()
    c += 1

index = 4
00110110
00110100

index = 11
01110010
01100010

index = 13
01100110
01100100

index = 34
01111011
01111001

index = 35
01110010
01111010

index = 64
00100111
00100011



In [197]:
with open('../datasets/languageDecryptor.json') as data_file:
    temp1 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp2 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp3 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp4 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp5 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp6 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp7 = json.load(data_file)

In [17]:
temp1

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0, 1, 1, 0, 0, 0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0, 0, 1, 1, 1, 0, 0, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0, 0, 1, 1, 1, 0, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 1, 1, 0, 0, 0, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 1, 1, 0, 0, 0, 1, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1, 1, 0, 0, 1, 0, 0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 0, 1, 1, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 1, 0, 0, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 0, 0

In [198]:
for i in range(100):
    temp1[i][-1] = temp1[i][-1][1:2]
    temp2[i][-1] = temp2[i][-1][2:3]
    temp3[i][-1] = temp3[i][-1][3:4]
    temp4[i][-1] = temp4[i][-1][4:5]
    temp5[i][-1] = temp5[i][-1][5:6]
    temp6[i][-1] = temp6[i][-1][6:7]
    temp7[i][-1] = temp7[i][-1][7:]

In [245]:
temp6

[[1, 1, 0, 1, 1, 1, 0, 1, [0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0]],
 [1, 1, 0, 0, 0, 1, 1, 0, [1]],
 [1, 0, 1, 1, 1, 1, 1, 0, [1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0]],
 [1, 0, 0, 1, 0, 1, 1, 0, [1]],
 [1, 0, 0, 0, 0, 1, 1, 0, [1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0]],
 [1, 0, 0, 1, 0, 1, 1, 1, [1]],
 [1, 0, 0, 0, 1, 1, 0, 1, [1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0]],
 [1, 0, 0, 0, 0, 0, 1, 1, [1]],
 [1, 0, 0, 1, 1, 1, 0, 0, [1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0]],
 [1, 0, 0, 0, 0, 0, 0, 0, [1]],
 [1, 1, 0, 1, 0, 0, 1, 0, [1]],
 [1, 1, 1, 1, 0, 1, 0, 1, [0]],
 [1, 1, 0, 1, 0, 1, 0, 1, [0]],
 [0, 0, 0, 1, 1, 0, 0, 0, [1]],
 [1, 1, 0, 1, 0, 0, 0, 1, [1]],
 [1, 0, 1, 1, 1, 1, 1, 1, [0]],
 [1, 1, 1, 1, 0, 0, 0, 1, [0]],
 [1, 0, 0, 1, 0, 1, 0, 1, [1]],
 [0, 0, 0, 1, 0, 1, 1, 0, [1]],
 [1, 1, 1, 0, 0, 1, 1, 0, [0]],
 [1, 1, 1, 1, 1, 1, 0, 1, [0]],
 [1, 0, 

In [151]:
# Read JSON file
with open('../network/network.json') as data_file:
    network2 = json.load(data_file)

In [260]:
seed(1)
network = initialize_network([8,10,9,1])

In [261]:

n_inputs = len(temp1[0]) - 1
n_outputs = len(temp1[0][-1])
start = timeit.default_timer()
train_network(network, temp7, 1.0, 10000, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

> epoch=99, lrate=1.000, error=25.324
> epoch=199, lrate=1.000, error=20.318
> epoch=299, lrate=1.000, error=6.957
> epoch=399, lrate=1.000, error=3.583
> epoch=499, lrate=1.000, error=2.834
> epoch=599, lrate=1.000, error=2.718
> epoch=699, lrate=1.000, error=2.583
> epoch=799, lrate=1.000, error=2.328
> epoch=899, lrate=1.000, error=0.044
> epoch=999, lrate=1.000, error=0.019
> epoch=1099, lrate=1.000, error=0.011
> epoch=1199, lrate=1.000, error=0.008
> epoch=1299, lrate=1.000, error=0.006
> epoch=1399, lrate=1.000, error=0.004
> epoch=1499, lrate=1.000, error=0.004
> epoch=1599, lrate=1.000, error=0.003
> epoch=1699, lrate=1.000, error=0.003
> epoch=1799, lrate=1.000, error=0.002
> epoch=1899, lrate=1.000, error=0.002
> epoch=1999, lrate=1.000, error=0.002
> epoch=2099, lrate=1.000, error=0.002
> epoch=2199, lrate=1.000, error=0.002
> epoch=2299, lrate=1.000, error=0.001
> epoch=2399, lrate=1.000, error=0.001
> epoch=2499, lrate=1.000, error=0.001
> epoch=2599, lrate=1.000, error=0

In [262]:
c = 0
new_dataset = list()
for i in temp7:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        new_dataset.append(i)
        c += 1
print(str(c)+" %")

0 %


In [264]:
# Write JSON file
with io.open('../network/decryptor/network_bit_8.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(network,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

In [127]:
seed(1)
network = initialize_network([8,9,7,2])

In [130]:

n_inputs = len(temp3[0]) - 1
n_outputs = len(temp3[0][-1])
start = timeit.default_timer()
train_network(network, temp3[0:50], 1.0, 10000, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

> epoch=99, lrate=1.000, error=1.001
> epoch=199, lrate=1.000, error=1.001
> epoch=299, lrate=1.000, error=1.000
> epoch=399, lrate=1.000, error=1.000
> epoch=499, lrate=1.000, error=1.000
> epoch=599, lrate=1.000, error=1.000
> epoch=699, lrate=1.000, error=1.000
> epoch=799, lrate=1.000, error=1.000
> epoch=899, lrate=1.000, error=1.000
> epoch=999, lrate=1.000, error=1.000
> epoch=1099, lrate=1.000, error=1.000
> epoch=1199, lrate=1.000, error=1.000
> epoch=1299, lrate=1.000, error=1.000
> epoch=1399, lrate=1.000, error=1.000
> epoch=1499, lrate=1.000, error=1.000
> epoch=1599, lrate=1.000, error=1.000
> epoch=1699, lrate=1.000, error=1.000
> epoch=1799, lrate=1.000, error=1.000
> epoch=1899, lrate=1.000, error=1.000
> epoch=1999, lrate=1.000, error=1.000
> epoch=2099, lrate=1.000, error=1.000
> epoch=2199, lrate=1.000, error=1.000
> epoch=2299, lrate=1.000, error=1.000
> epoch=2399, lrate=1.000, error=1.000
> epoch=2499, lrate=1.000, error=1.000
> epoch=2599, lrate=1.000, error=1.0

In [129]:
c = 0
new_dataset = list()
for i in temp3:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        new_dataset.append(i)
        c += 1
print(str(c)+" %")

3 %


In [45]:
# Write JSON file
with io.open('../network/decryptor/network2.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(network,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

In [265]:
# Read JSON file
with open('../network/decryptor/network_bit_2.json') as data_file:
    network2 = json.load(data_file)
    
with open('../network/decryptor/network_bit_3.json') as data_file:
    network3 = json.load(data_file)
    
with open('../network/decryptor/network_bit_4.json') as data_file:
    network4 = json.load(data_file)
    
with open('../network/decryptor/network_bit_5.json') as data_file:
    network5 = json.load(data_file)
    
with open('../network/decryptor/network_bit_6.json') as data_file:
    network6 = json.load(data_file)
    
with open('../network/decryptor/network_bit_7.json') as data_file:
    network7 = json.load(data_file)
    
with open('../network/decryptor/network_bit_8.json') as data_file:
    network8 = json.load(data_file)
   
